In [11]:
import os
from dotenv import load_dotenv
import requests
from web3 import Web3, HTTPProvider
import pandas as pd

# Load .env file
load_dotenv()

True

In [5]:
# Setup Reddit API
auth = requests.auth.HTTPBasicAuth(os.getenv('REDDIT_CLIENT_ID'), os.getenv('REDDIT_CLIENT_SECRET'))

data = {
    'grant_type': 'password',
    'username': os.getenv('REDDIT_USERNAME'),
    'password': os.getenv('REDDIT_PASSWORD')
}

header = {
    'User-Agent': 'MyAPI/0.0.1'
}

res = requests.post(
    'https://www.reddit.com/api/v1/access_token', 
    auth=auth, 
    data=data, 
    headers=header
)

TOKEN = res.json()['access_token']

headers = {
    **header,
    'Authorization': f'bearer {TOKEN}'
}

In [7]:
def get_reddi_data(contract_name, token_name, token_symbol, limit=10):
    # Specify the search query with hashtags
    search_query = f"{contract_name} OR {token_name} AND {token_symbol}"

    # Parameters for the request
    params = {
        "q": search_query,
        "sort": "relevance",
        "limit": limit
    }
    res = requests.get(
        f'https://oauth.reddit.com/r/{os.getenv("SUBR")}/search.json',
        params=params,
        headers=headers
    )

    return res.json()

In [8]:
# Use etherscan api to extract contract constructor name
def get_contract_name(address):
    etherscan_api_key = os.getenv('ETHERSCAN_API_KEY')
    etherscan_url = f'https://api.etherscan.io/api?module=contract&action=getsourcecode&address={address}&apikey={etherscan_api_key}'
    
    try:
        response = requests.get(etherscan_url)
        data = response.json()
        if data.get('status') == '1' and data.get('result'):
            contract_name = data['result'][0]['ContractName']
            return contract_name
        else:
            return "Contract name not found"
    except Exception as e:
        return f"Error fetching contract name: {str(e)}"

In [9]:
def get_token_name_and_symbol(address):
    etherscan_api_key = os.getenv('ETHERSCAN_API_KEY')
    etherscan_url = f'https://api.etherscan.io/api?module=contract&action=getsourcecode&address={address}&apikey={etherscan_api_key}'

    try:
        infura_api_key = os.getenv('INFURA_API_KEY')
        response = requests.get(etherscan_url)
        data = response.json()
        if data.get('status') == '1' and data.get('result'):
            contract_abi = data['result'][0]['ABI']
            contract_address = address  # Address of the deployed contract

            # Connect to the Ethereum network using Web3
            web3 = Web3(HTTPProvider(f'https://mainnet.infura.io/v3/{infura_api_key}'))  # Replace with your Infura project ID

            # Create a contract instance
            contract = web3.eth.contract(address=contract_address, abi=contract_abi)

            # Call the 'name' function of the ERC20 contract to get the token name
            token_name = contract.functions.name().call()
            token_symbol = contract.functions.symbol().call()

            return token_name, token_symbol
        else:
            return "Token name not found"
    except Exception as e:
        return f"Error fetching token name: {str(e)}"

In [10]:
uni_contract = "0x1f9840a85d5aF5bf1D1762F925BDADdC4201F984"

# Get contract name
contract_name = get_contract_name(uni_contract)

# Get toke name and symbol
token_name, token_symbol = get_token_name_and_symbol(uni_contract)

# Get reddit data
reddit_data = get_reddi_data(contract_name, token_name, token_symbol)

print(reddit_data)

{'kind': 'Listing', 'data': {'modhash': None, 'dist': 10, 'facets': {}, 'after': 't3_15x6z4a', 'geo_filter': '', 'children': [{'kind': 't3', 'data': {'approved_at_utc': None, 'subreddit': 'CryptoCurrency', 'selftext': '', 'author_fullname': 't2_ax5ierpr', 'saved': False, 'mod_reason_title': None, 'gilded': 0, 'clicked': False, 'title': "UniSwap's Recent Decision Becomes a Point of Controversy: What's the Truth of the KYC and Whitelist Debate?", 'link_flair_richtext': [{'e': 'text', 't': 'GENERAL-NEWS'}], 'subreddit_name_prefixed': 'r/CryptoCurrency', 'hidden': False, 'pwls': 6, 'link_flair_css_class': 'NEWS', 'downs': 0, 'thumbnail_height': 70, 'top_awarded_type': None, 'hide_score': False, 'name': 't3_178j86e', 'quarantine': False, 'link_flair_text_color': 'light', 'upvote_ratio': 0.68, 'author_flair_background_color': '', 'ups': 20, 'total_awards_received': 0, 'media_embed': {}, 'thumbnail_width': 140, 'author_flair_template_id': None, 'is_original_content': False, 'user_reports': []

In [13]:
# Get titles and content from reddit data
titles = [post['data']['title'] for post in reddit_data['data']['children']]
contents = [post['data']['selftext'] for post in reddit_data['data']['children']]

In [14]:
# Convert titles and content to pandas dataframe
df = pd.DataFrame({'title': titles, 'content': contents})

In [15]:
df

,title,content
0,UniSwap's Recent Decision Becomes a Point of C...,
1,"How much will the price of UNI move by Monday,...",Reference price: $36.20\n\n[Data will be sourc...
2,How will crypto be taken seriously if major pl...,We keep preaching about “crypto becoming mains...
3,"💥 UniSwap V4 💥 More innovation, MEV protection...",[UniSwap blog - V4 vision](https://blog.uniswa...
4,"UNI is now on Nexo! You can buy, exchange, and...",
5,Will Uniswap (UNI) Implement KYC?,
6,What is the UniSwap equivalent on Cardano?,
7,MemeCoin Season is Here! FlokiGF - $FlokiGF 🧡 ...,FlokiGF - $FLOKIGF 🧡\n\n&amp;#x200B;\n\nFlokiG...
8,Uni(TRADE) -- limit orders and automated tradi...,This project will be a game changer for all us...
9,UniSwap V4 threat modeling,Interesting article for anyone planning to int...
